In [28]:
from datetime import datetime
import time
import win32com.client
import os
import pyautogui
from openpyxl import load_workbook,Workbook

In [2]:
 pip install Pillow

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [29]:
def get_session():
    try:
        # Abrir instancia en SAP
        SapGuiAuto = win32com.client.GetObject("SAPGUI")
        app = SapGuiAuto.GetScriptingEngine
        connection = app.Children(0)
        session = connection.Children(0)
        return session
    except Exception as e:
        print("Error GET_SESSION:", str(e))

def enter_tx(tx):
    global session
    try:
        session.findById("wnd[0]/tbar[0]/okcd").text = f"/n{tx}"
        session.findById("wnd[0]").sendVKey(0)
    except Exception as e:
        print("Error ENTER_TX:", str(e))

In [25]:
# Cargar archivo de asientos
archivo = r"C:\Users\Ext_rgutierrezc\OneDrive - Grupo Alicorp\Escritorio\Siglo BPO (1)\Base2.xlsx"
wb = load_workbook(archivo)
ws=wb['DATA']
sociedad="PE18"

# Definir la ruta de la carpeta donde se guardarán las capturas de pantalla
base_path = r"C:\Users\Ext_rgutierrezc\OneDrive - Grupo Alicorp\Escritorio\Siglo BPO (1)"
ss_folder = os.path.join(base_path, "SCREENSHOTS")

# Crear la carpeta "SS" si no existe
if not os.path.exists(ss_folder):
    os.makedirs(ss_folder)

# Esto para extraer el material

In [7]:

# Cargar archivo de asientos
archivo = r"C:\Users\Ext_rgutierrezc\OneDrive - Grupo Alicorp\Escritorio\Siglo BPO (1)\BASEMATERIAL.xlsx"
wb = load_workbook(archivo)
ws = wb['DATA']
sociedad = "PE18"

session = get_session()

# Identificar las columnas en las que quieres guardar los datos adicionales
columna_montoSAP = ws.max_column + 1  # Primera columna vacía para guardar montoSAP
columna_material = columna_montoSAP + 1  # Siguiente columna para guardar material

# Escribir encabezados para las nuevas columnas
ws.cell(row=1, column=columna_montoSAP).value = "Monto SAP"
ws.cell(row=1, column=columna_material).value = "Material"

for i, fila in enumerate(ws.iter_rows(min_row=2, max_row=ws.max_row, values_only=True), start=2):
    # Solo filas con datos
    nro_documento = fila[0]
    sociedad = "PE18"
    montoINPUT = fila[1]  # Columna B según tu descripción

    # Si no hay número de documento, saltar a la siguiente fila
    if not nro_documento:
        continue

    try:
        ejercicio = 2023
        # Ir a la TX 
        enter_tx("FB03")
        session.findById("wnd[0]").maximize()

        # Nro Documento
        session.findById("wnd[0]/usr/txtRF05L-BELNR").text = nro_documento
        # Sociedad
        session.findById("wnd[0]/usr/ctxtRF05L-BUKRS").text = sociedad
        # Ejercicio
        session.findById("wnd[0]/usr/txtRF05L-GJAHR").text = ejercicio
        # Ejecutar
        session.findById("wnd[0]").sendVKey(0)

        # FILTRAR     
        session.findById("wnd[0]/usr/cntlCTRL_CONTAINERBSEG/shellcont/shell").setCurrentCell(-1, "AZBET")
        session.findById("wnd[0]/usr/cntlCTRL_CONTAINERBSEG/shellcont/shell").selectColumn("AZBET")
        session.findById("wnd[0]/usr/cntlCTRL_CONTAINERBSEG/shellcont/shell").selectedRows = ""
        session.findById("wnd[0]/usr/cntlCTRL_CONTAINERBSEG/shellcont/shell").pressToolbarButton("&MB_FILTER")
        session.findById("wnd[1]/usr/ssub%_SUBSCREEN_FREESEL:SAPLSSEL:1105/txt%%DYN002-LOW").text = montoINPUT
        session.findById("wnd[1]/usr/ssub%_SUBSCREEN_FREESEL:SAPLSSEL:1105/txt%%DYN002-LOW").setFocus()
        session.findById("wnd[1]/usr/ssub%_SUBSCREEN_FREESEL:SAPLSSEL:1105/txt%%DYN002-LOW").caretPosition = 7
        session.findById("wnd[1]/tbar[0]/btn[0]").press()

        # CLICK EN EL 1ERO 
        session.findById("wnd[0]/usr/cntlCTRL_CONTAINERBSEG/shellcont/shell").currentCellColumn = "DMBE2"
        session.findById("wnd[0]/usr/cntlCTRL_CONTAINERBSEG/shellcont/shell").selectedRows = "0"
        session.findById("wnd[0]/usr/cntlCTRL_CONTAINERBSEG/shellcont/shell").doubleClickCurrentCell()
        
        # SACAR LA INFO
        montoSAP = session.findById("wnd[0]/usr/txtBSEG-WRBTR").text
        
        # EL MAS    
        session.findById("wnd[0]/usr/subBLOCK:SAPLKACB:1014/btnCOBL_MORE").press()
        
        # SACAR LA INFO 
        material = session.findById("wnd[1]/usr/ctxtCOBL-MATNR").text
        session.findById("wnd[1]").close()

        # Guardar las variables en las columnas correspondientes
        ws.cell(row=i, column=columna_montoSAP).value = montoSAP
        ws.cell(row=i, column=columna_material).value = material

    except Exception as e:
        # En caso de error, almacenar el error en la columna final
        ws.cell(row=i, column=ws.max_column + 1).value = f"Error: {str(e)}"
        print(f"Error o fin de ítems en el ítem {i - 1} del documento {nro_documento}: {str(e)}")

# Finalmente, guardar el archivo Excel con los cambios
wb.save(r"C:\Users\Ext_rgutierrezc\OneDrive - Grupo Alicorp\Escritorio\Siglo BPO (1)\BASEMATERIAL_actualizado.xlsx")


In [17]:
session = get_session()

for i, fila in enumerate(ws.iter_rows(min_row=2, max_row=ws.max_row, values_only=True), start=2):
    # Solo filas con datos
    nro_documento = fila[0]
    sociedad="PE18"
    if not nro_documento:
        continue

    try:
        ejercicio = 2022
        # Ir a la TX 
        enter_tx("FB03")
        session.findById("wnd[0]").maximize()

        # Nro Documento
        session.findById("wnd[0]/usr/txtRF05L-BELNR").text = nro_documento
        # Sociedad
        session.findById("wnd[0]/usr/ctxtRF05L-BUKRS").text = sociedad
        # Ejercicio
        session.findById("wnd[0]/usr/txtRF05L-GJAHR").text = ejercicio
        # Ejecutar
        session.findById("wnd[0]").sendVKey(0)

        ### SOMBRERITO

        session.findById("wnd[0]/tbar[1]/btn[5]").press()
        session.findById("wnd[1]/usr/txtBKPF-AWKEY").setFocus()
        session.findById("wnd[1]/usr/txtBKPF-AWKEY").caretPosition = 14
        session.findById("wnd[1]").sendVKey(2)

        # SELECCIONAR CANTIDADES
        session.findById("wnd[0]/usr/ssubSUB_MAIN_CARRIER:SAPLMIGO:0003/subSUB_ITEMDETAIL:SAPLMIGO:0301/subSUB_DETAIL:SAPLMIGO:0300/tabsTS_GOITEM/tabpOK_GOITEM_QUANTITIES").select()

        # Intentamos seleccionar los ítems manualmente uno por uno
        try:
            # Ítem 0
            session.findById("wnd[0]/usr/ssubSUB_MAIN_CARRIER:SAPLMIGO:0003/subSUB_ITEMLIST:SAPLMIGO:0200/tblSAPLMIGOTV_GOITEM").columns.elementAt(2).selected = True
            session.findById("wnd[0]/usr/ssubSUB_MAIN_CARRIER:SAPLMIGO:0003/subSUB_ITEMLIST:SAPLMIGO:0200/tblSAPLMIGOTV_GOITEM").reorderTable("0 45 1 46 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77")
            session.findById("wnd[0]/usr/ssubSUB_MAIN_CARRIER:SAPLMIGO:0003/subSUB_ITEMLIST:SAPLMIGO:0200/tblSAPLMIGOTV_GOITEM").columns.elementAt(2).width = 38
            session.findById("wnd[0]/usr/ssubSUB_MAIN_CARRIER:SAPLMIGO:0003/subSUB_ITEMLIST:SAPLMIGO:0200/tblSAPLMIGOTV_GOITEM").columns.elementAt(1).width = 11
            session.findById("wnd[0]/usr/ssubSUB_MAIN_CARRIER:SAPLMIGO:0003/subSUB_ITEMLIST:SAPLMIGO:0200/tblSAPLMIGOTV_GOITEM/btnGOITEM-ZEILE[0,0]").setFocus()
            session.findById("wnd[0]/usr/ssubSUB_MAIN_CARRIER:SAPLMIGO:0003/subSUB_ITEMLIST:SAPLMIGO:0200/tblSAPLMIGOTV_GOITEM/btnGOITEM-ZEILE[0,0]").press()
            time.sleep(1)
            screenshot = pyautogui.screenshot()
             # Convertir la captura de pantalla a un objeto Pillow para poder modificarla
            
        # Redimensionar la captura (por ejemplo, a 800x600 píxeles)
            
            screenshot.save(os.path.join(ss_folder, f"{nro_documento}-0.png"))
                        # Ítem 1

            session.findById("wnd[0]/usr/ssubSUB_MAIN_CARRIER:SAPLMIGO:0003/subSUB_ITEMLIST:SAPLMIGO:0200/tblSAPLMIGOTV_GOITEM/btnGOITEM-ZEILE[0,1]").setFocus()
            session.findById("wnd[0]/usr/ssubSUB_MAIN_CARRIER:SAPLMIGO:0003/subSUB_ITEMLIST:SAPLMIGO:0200/tblSAPLMIGOTV_GOITEM/btnGOITEM-ZEILE[0,1]").press()
            time.sleep(1)
            screenshot = pyautogui.screenshot()
             # Convertir la captura de pantalla a un objeto Pillow para poder modificarla
            
        # Redimensionar la captura (por ejemplo, a 800x600 píxeles)
            
            screenshot.save(os.path.join(ss_folder, f"{nro_documento}-1.png"))
                    # Ítem 2
            session.findById("wnd[0]/usr/ssubSUB_MAIN_CARRIER:SAPLMIGO:0003/subSUB_ITEMLIST:SAPLMIGO:0200/tblSAPLMIGOTV_GOITEM/btnGOITEM-ZEILE[0,2]").setFocus()
            session.findById("wnd[0]/usr/ssubSUB_MAIN_CARRIER:SAPLMIGO:0003/subSUB_ITEMLIST:SAPLMIGO:0200/tblSAPLMIGOTV_GOITEM/btnGOITEM-ZEILE[0,2]").press()
            time.sleep(1)
            screenshot = pyautogui.screenshot()
             # Convertir la captura de pantalla a un objeto Pillow para poder modificarla
            
        # Redimensionar la captura (por ejemplo, a 800x600 píxeles)
            
            screenshot.save(os.path.join(ss_folder, f"{nro_documento}-2.png"))
                        # Ítem 3
            session.findById("wnd[0]/usr/ssubSUB_MAIN_CARRIER:SAPLMIGO:0003/subSUB_ITEMLIST:SAPLMIGO:0200/tblSAPLMIGOTV_GOITEM/btnGOITEM-ZEILE[0,3]").setFocus()
            session.findById("wnd[0]/usr/ssubSUB_MAIN_CARRIER:SAPLMIGO:0003/subSUB_ITEMLIST:SAPLMIGO:0200/tblSAPLMIGOTV_GOITEM/btnGOITEM-ZEILE[0,3]").press()
            time.sleep(1)
            screenshot = pyautogui.screenshot()
                # Convertir la captura de pantalla a un objeto Pillow para poder modificarla
            
        # Redimensionar la captura (por ejemplo, a 800x600 píxeles)
            
            screenshot.save(os.path.join(ss_folder, f"{nro_documento}-3.png"))
                        # Ítem 4
            session.findById("wnd[0]/usr/ssubSUB_MAIN_CARRIER:SAPLMIGO:0003/subSUB_ITEMLIST:SAPLMIGO:0200/tblSAPLMIGOTV_GOITEM").verticalScrollbar.position = 4
            time.sleep(3)
            session.findById("wnd[0]/usr/ssubSUB_MAIN_CARRIER:SAPLMIGO:0003/subSUB_ITEMLIST:SAPLMIGO:0200/tblSAPLMIGOTV_GOITEM/btnGOITEM-ZEILE[0,0]").setFocus()
            session.findById("wnd[0]/usr/ssubSUB_MAIN_CARRIER:SAPLMIGO:0003/subSUB_ITEMLIST:SAPLMIGO:0200/tblSAPLMIGOTV_GOITEM/btnGOITEM-ZEILE[0,0]").press()
            time.sleep(1)
            screenshot = pyautogui.screenshot()
            
            screenshot.save(os.path.join(ss_folder, f"{nro_documento}-4.png"))
                        # Ítem 5

            session.findById("wnd[0]/usr/ssubSUB_MAIN_CARRIER:SAPLMIGO:0003/subSUB_ITEMLIST:SAPLMIGO:0200/tblSAPLMIGOTV_GOITEM/btnGOITEM-ZEILE[0,1]").setFocus()
            session.findById("wnd[0]/usr/ssubSUB_MAIN_CARRIER:SAPLMIGO:0003/subSUB_ITEMLIST:SAPLMIGO:0200/tblSAPLMIGOTV_GOITEM/btnGOITEM-ZEILE[0,1]").press()
            time.sleep(1)
            screenshot = pyautogui.screenshot()
             # Convertir la captura de pantalla a un objeto Pillow para poder modificarla
            
        # Redimensionar la captura (por ejemplo, a 800x600 píxeles)
            
            screenshot.save(os.path.join(ss_folder, f"{nro_documento}-5.png"))
            # Ítem 6
            session.findById("wnd[0]/usr/ssubSUB_MAIN_CARRIER:SAPLMIGO:0003/subSUB_ITEMLIST:SAPLMIGO:0200/tblSAPLMIGOTV_GOITEM/btnGOITEM-ZEILE[0,2]").setFocus()
            session.findById("wnd[0]/usr/ssubSUB_MAIN_CARRIER:SAPLMIGO:0003/subSUB_ITEMLIST:SAPLMIGO:0200/tblSAPLMIGOTV_GOITEM/btnGOITEM-ZEILE[0,2]").press()
            time.sleep(1)
            screenshot = pyautogui.screenshot()
             # Convertir la captura de pantalla a un objeto Pillow para poder modificarla
            
        # Redimensionar la captura (por ejemplo, a 800x600 píxeles)
            
            screenshot.save(os.path.join(ss_folder, f"{nro_documento}-6.png"))
                        # Ítem 7
            session.findById("wnd[0]/usr/ssubSUB_MAIN_CARRIER:SAPLMIGO:0003/subSUB_ITEMLIST:SAPLMIGO:0200/tblSAPLMIGOTV_GOITEM/btnGOITEM-ZEILE[0,3]").setFocus()
            session.findById("wnd[0]/usr/ssubSUB_MAIN_CARRIER:SAPLMIGO:0003/subSUB_ITEMLIST:SAPLMIGO:0200/tblSAPLMIGOTV_GOITEM/btnGOITEM-ZEILE[0,3]").press()
            time.sleep(1)
            screenshot = pyautogui.screenshot()
             # Convertir la captura de pantalla a un objeto Pillow para poder modificarla
            
        # Redimensionar la captura (por ejemplo, a 800x600 píxeles)
            
            screenshot.save(os.path.join(ss_folder, f"{nro_documento}-7.png"))
                    # Ítem 2
            session.findById("wnd[0]/usr/ssubSUB_MAIN_CARRIER:SAPLMIGO:0003/subSUB_ITEMLIST:SAPLMIGO:0200/tblSAPLMIGOTV_GOITEM").verticalScrollbar.position = 8
            time.sleep(3)
            session.findById("wnd[0]/usr/ssubSUB_MAIN_CARRIER:SAPLMIGO:0003/subSUB_ITEMLIST:SAPLMIGO:0200/tblSAPLMIGOTV_GOITEM/btnGOITEM-ZEILE[0,0]").setFocus()
            session.findById("wnd[0]/usr/ssubSUB_MAIN_CARRIER:SAPLMIGO:0003/subSUB_ITEMLIST:SAPLMIGO:0200/tblSAPLMIGOTV_GOITEM/btnGOITEM-ZEILE[0,0]").press()
            time.sleep(1)
            screenshot = pyautogui.screenshot()
              # Convertir la captura de pantalla a un objeto Pillow para poder modificarla
            
        # Redimensionar la captura (por ejemplo, a 800x600 píxeles)
            
            screenshot.save(os.path.join(ss_folder, f"{nro_documento}-8.png"))
                        # Ítem 3
  
            session.findById("wnd[0]/usr/ssubSUB_MAIN_CARRIER:SAPLMIGO:0003/subSUB_ITEMLIST:SAPLMIGO:0200/tblSAPLMIGOTV_GOITEM/btnGOITEM-ZEILE[0,1]").setFocus()
            session.findById("wnd[0]/usr/ssubSUB_MAIN_CARRIER:SAPLMIGO:0003/subSUB_ITEMLIST:SAPLMIGO:0200/tblSAPLMIGOTV_GOITEM/btnGOITEM-ZEILE[0,1]").press()
            time.sleep(1)
            screenshot = pyautogui.screenshot()
                # Convertir la captura de pantalla a un objeto Pillow para poder modificarla
            
        # Redimensionar la captura (por ejemplo, a 800x600 píxeles)
            
            screenshot.save(os.path.join(ss_folder, f"{nro_documento}-9.png"))
                        # Ítem 4
            session.findById("wnd[0]/usr/ssubSUB_MAIN_CARRIER:SAPLMIGO:0003/subSUB_ITEMLIST:SAPLMIGO:0200/tblSAPLMIGOTV_GOITEM/btnGOITEM-ZEILE[0,2]").setFocus()
            session.findById("wnd[0]/usr/ssubSUB_MAIN_CARRIER:SAPLMIGO:0003/subSUB_ITEMLIST:SAPLMIGO:0200/tblSAPLMIGOTV_GOITEM/btnGOITEM-ZEILE[0,2]").press()
            time.sleep(1)
            screenshot = pyautogui.screenshot()
            
        # Redimensionar la captura (por ejemplo, a 800x600 píxeles)
                      
            screenshot.save(os.path.join(ss_folder, f"{nro_documento}-10.png"))
                        # Ítem 5
            session.findById("wnd[0]/usr/ssubSUB_MAIN_CARRIER:SAPLMIGO:0003/subSUB_ITEMLIST:SAPLMIGO:0200/tblSAPLMIGOTV_GOITEM/btnGOITEM-ZEILE[0,3]").setFocus()
            session.findById("wnd[0]/usr/ssubSUB_MAIN_CARRIER:SAPLMIGO:0003/subSUB_ITEMLIST:SAPLMIGO:0200/tblSAPLMIGOTV_GOITEM/btnGOITEM-ZEILE[0,3]").press()
            time.sleep(1)
            screenshot = pyautogui.screenshot()
            
        # Redimensionar la captura (por ejemplo, a 800x600 píxeles)
            screenshot.save(os.path.join(ss_folder, f"{nro_documento}-11.png"))

        except Exception as e:
            print(f"Error o fin de ítems en el ítem 0 del documento {nro_documento}: {str(e)}")


    except Exception as e:
        # En caso de error, almacenar el error en la columna final
        ws.cell(row=i, column=ws.max_column).value = f"Error: {str(e)}"

# Finalmente, guardar el archivo Excel con los cambios
wb.save("ruta_al_archivo.xlsx")


# Sacar primer pantallazo

In [13]:
# Cargar archivo de asientos
archivo = r"C:\Users\Ext_rgutierrezc\OneDrive - Grupo Alicorp\Escritorio\Siglo BPO (1)\Base2.xlsx"
wb = load_workbook(archivo)
ws=wb['DATA']
sociedad="PE18"

# Definir la ruta de la carpeta donde se guardarán las capturas de pantalla
base_path = r"C:\Users\Ext_rgutierrezc\OneDrive - Grupo Alicorp\Escritorio\Siglo BPO (1)"
ss_folder = os.path.join(base_path, "PRIMER - SCREENSHOTS - MILAGROS")

# Crear la carpeta "SS" si no existe
if not os.path.exists(ss_folder):
    os.makedirs(ss_folder)

In [15]:
session = get_session()

for i, fila in enumerate(ws.iter_rows(min_row=2, max_row=ws.max_row, values_only=True), start=2):
    # Solo filas con datos
    nro_documento = fila[0]
    monto_positivo = fila[1]
    monto_negativo = fila[2]

    sociedad="PE18"
    if not nro_documento:
        continue

    try:
        ejercicio = 2022
        # Ir a la TX 
        enter_tx("FB03")
        session.findById("wnd[0]").maximize()

        # Nro Documento
        session.findById("wnd[0]/usr/txtRF05L-BELNR").text = nro_documento
        # Sociedad
        session.findById("wnd[0]/usr/ctxtRF05L-BUKRS").text = sociedad
        # Ejercicio
        session.findById("wnd[0]/usr/txtRF05L-GJAHR").text = ejercicio
        # Ejecutar
        session.findById("wnd[0]").sendVKey(0)

     ### ASIENTO MERMAS - CAMBIO LAYOUT
        session.findById("wnd[0]/usr/cntlCTRL_CONTAINERBSEG/shellcont/shell").pressToolbarButton("&MB_VARIANT")
        session.findById("wnd[1]/usr/subSUB_CONFIGURATION:SAPLSALV_CUL_LAYOUT_CHOOSE:0500/cntlD500_CONTAINER/shellcont/shell").currentCellRow = 6
        session.findById("wnd[1]/usr/subSUB_CONFIGURATION:SAPLSALV_CUL_LAYOUT_CHOOSE:0500/cntlD500_CONTAINER/shellcont/shell").selectedRows = "6"
        session.findById("wnd[1]/usr/subSUB_CONFIGURATION:SAPLSALV_CUL_LAYOUT_CHOOSE:0500/cntlD500_CONTAINER/shellcont/shell").clickCurrentCell()
    #Seleccionar columna a filtrar y filtrar

        session.findById("wnd[0]/usr/cntlCTRL_CONTAINERBSEG/shellcont/shell").setCurrentCell(-1, "DMBTR")
        session.findById("wnd[0]/usr/cntlCTRL_CONTAINERBSEG/shellcont/shell").selectColumn("DMBTR")
        session.findById("wnd[0]/usr/cntlCTRL_CONTAINERBSEG/shellcont/shell").pressToolbarButton("&MB_FILTER")

    # Ventanita para colocar los límites

        session.findById("wnd[1]/usr/ssub%_SUBSCREEN_FREESEL:SAPLSSEL:1105/btn%_%%DYN002_%_APP_%-VALU_PUSH").press()

    #Colocas los montos

        session.findById("wnd[2]/usr/tabsTAB_STRIP/tabpSIVA/ssubSCREEN_HEADER:SAPLALDB:3010/tblSAPLALDBSINGLE/txtRSCSEL_255-SLOW_I[1,0]").text = monto_positivo
        session.findById("wnd[2]/usr/tabsTAB_STRIP/tabpSIVA/ssubSCREEN_HEADER:SAPLALDB:3010/tblSAPLALDBSINGLE/txtRSCSEL_255-SLOW_I[1,1]").text = monto_negativo
        session.findById("wnd[2]/usr/tabsTAB_STRIP/tabpSIVA/ssubSCREEN_HEADER:SAPLALDB:3010/tblSAPLALDBSINGLE/txtRSCSEL_255-SLOW_I[1,1]").setFocus()
        session.findById("wnd[2]/usr/tabsTAB_STRIP/tabpSIVA/ssubSCREEN_HEADER:SAPLALDB:3010/tblSAPLALDBSINGLE/txtRSCSEL_255-SLOW_I[1,1]").caretPosition = 7
        session.findById("wnd[2]/tbar[0]/btn[8]").press()
        session.findById("wnd[1]/tbar[0]/btn[0]").press()


    # Definir las coordenadas y el tamaño de la región para la captura de pantalla
        x, y, width, height = 0, 150, 1200, 400  # Ajusta estos valores según lo que necesites
    #Tomar foto
        time.sleep(1)
        screenshot = pyautogui.screenshot(region=(x, y, width, height))
    #Guardar la captura      
        screenshot.save(os.path.join(ss_folder, f"{nro_documento}-{monto_positivo}.png"))        

    except Exception as e:
        # En caso de error, almacenar el error en la columna final
        ws.cell(row=i, column=ws.max_column).value = f"Error: {str(e)}"

# Finalmente, guardar el archivo Excel con los cambios
wb.save("ruta_al_archivo.xlsx")


# SACAR 2DO PANTALLAZO ESTRUCTURA DIFERENTE

In [8]:
# Cargar archivo de asientos
archivo = r"C:\Users\Ext_rgutierrezc\OneDrive - Grupo Alicorp\Escritorio\Siglo BPO (1)\Base2.xlsx"
wb = load_workbook(archivo)
ws=wb['DATA']
sociedad="PE18"

# Definir la ruta de la carpeta donde se guardarán las capturas de pantalla
base_path = r"C:\Users\Ext_rgutierrezc\OneDrive - Grupo Alicorp\Escritorio\Siglo BPO (1)"
ss_folder = os.path.join(base_path, "SCREENSHOTS")

# Crear la carpeta "SS" si no existe
if not os.path.exists(ss_folder):
    os.makedirs(ss_folder)

PermissionError: [Errno 13] Permission denied: 'C:\\Users\\Ext_rgutierrezc\\OneDrive - Grupo Alicorp\\Escritorio\\Siglo BPO (1)\\Base2.xlsx'

In [31]:
session = get_session()

for i, fila in enumerate(ws.iter_rows(min_row=2, max_row=ws.max_row, values_only=True), start=2):
    # Solo filas con datos
    nro_documento = fila[0]
    sociedad="PE18"
    if not nro_documento:
        continue

    try:
        ejercicio = 2022
        # Ir a la TX 
        enter_tx("FB03")
        session.findById("wnd[0]").maximize()

        # Nro Documento
        session.findById("wnd[0]/usr/txtRF05L-BELNR").text = nro_documento
        # Sociedad
        session.findById("wnd[0]/usr/ctxtRF05L-BUKRS").text = sociedad
        # Ejercicio
        session.findById("wnd[0]/usr/txtRF05L-GJAHR").text = ejercicio
        # Ejecutar
        session.findById("wnd[0]").sendVKey(0)

        ### SOMBRERITO

        session.findById("wnd[0]/tbar[1]/btn[5]").press()
        session.findById("wnd[1]/usr/txtBKPF-AWKEY").setFocus()
        session.findById("wnd[1]/usr/txtBKPF-AWKEY").caretPosition = 14
        session.findById("wnd[1]").sendVKey(2)

        # SELECCIONAR CANTIDADES
        session.findById("wnd[0]/usr/ssubSUB_MAIN_CARRIER:SAPLMIGO:0003/subSUB_ITEMDETAIL:SAPLMIGO:0301/subSUB_DETAIL:SAPLMIGO:0300/tabsTS_GOITEM/tabpOK_GOITEM_QUANTITIES").select()

        # Intentamos seleccionar los ítems manualmente uno por uno
        try:
            # Ítem 0
            session.findById("wnd[0]/usr/ssubSUB_MAIN_CARRIER:SAPLMIGO:0003/subSUB_ITEMLIST:SAPLMIGO:0200/tblSAPLMIGOTV_GOITEM").columns.elementAt(2).selected = True
            session.findById("wnd[0]/usr/ssubSUB_MAIN_CARRIER:SAPLMIGO:0003/subSUB_ITEMLIST:SAPLMIGO:0200/tblSAPLMIGOTV_GOITEM").reorderTable("0 45 1 46 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77")
            session.findById("wnd[0]/usr/ssubSUB_MAIN_CARRIER:SAPLMIGO:0003/subSUB_ITEMLIST:SAPLMIGO:0200/tblSAPLMIGOTV_GOITEM").columns.elementAt(2).width = 38
            session.findById("wnd[0]/usr/ssubSUB_MAIN_CARRIER:SAPLMIGO:0003/subSUB_ITEMLIST:SAPLMIGO:0200/tblSAPLMIGOTV_GOITEM").columns.elementAt(1).width = 11

            session.findById("wnd[0]/usr/ssubSUB_MAIN_CARRIER:SAPLMIGO:0003/subSUB_ITEMLIST:SAPLMIGO:0200/tblSAPLMIGOTV_GOITEM/btnGOITEM-ZEILE[0,0]").setFocus()
            session.findById("wnd[0]/usr/ssubSUB_MAIN_CARRIER:SAPLMIGO:0003/subSUB_ITEMLIST:SAPLMIGO:0200/tblSAPLMIGOTV_GOITEM/btnGOITEM-ZEILE[0,0]").press()
            time.sleep(1)
            screenshot = pyautogui.screenshot()

            #Filtro para ver si es el ultimo ya alamos
            session.findById("wnd[0]/usr/ssubSUB_MAIN_CARRIER:SAPLMIGO:0003/subSUB_ITEMLIST:SAPLMIGO:0200/tblSAPLMIGOTV_GOITEM/btnGOITEM-ZEILE[0,1]").setFocus()
            session.findById("wnd[0]/usr/ssubSUB_MAIN_CARRIER:SAPLMIGO:0003/subSUB_ITEMLIST:SAPLMIGO:0200/tblSAPLMIGOTV_GOITEM/btnGOITEM-ZEILE[0,1]").press()
            
            screenshot.save(os.path.join(ss_folder, f"{nro_documento}-0.png"))
                        # Ítem 1
            session.findById("wnd[0]/usr/ssubSUB_MAIN_CARRIER:SAPLMIGO:0003/subSUB_ITEMLIST:SAPLMIGO:0200/tblSAPLMIGOTV_GOITEM").verticalScrollbar.position = 1
            
            session.findById("wnd[0]/usr/ssubSUB_MAIN_CARRIER:SAPLMIGO:0003/subSUB_ITEMLIST:SAPLMIGO:0200/tblSAPLMIGOTV_GOITEM/btnGOITEM-ZEILE[0,0]").setFocus()
            session.findById("wnd[0]/usr/ssubSUB_MAIN_CARRIER:SAPLMIGO:0003/subSUB_ITEMLIST:SAPLMIGO:0200/tblSAPLMIGOTV_GOITEM/btnGOITEM-ZEILE[0,0]").press()
            time.sleep(1)
            screenshot = pyautogui.screenshot()
             
            #Filtro para ver si es el ultimo ya alamos
            session.findById("wnd[0]/usr/ssubSUB_MAIN_CARRIER:SAPLMIGO:0003/subSUB_ITEMLIST:SAPLMIGO:0200/tblSAPLMIGOTV_GOITEM/btnGOITEM-ZEILE[0,1]").setFocus()
            session.findById("wnd[0]/usr/ssubSUB_MAIN_CARRIER:SAPLMIGO:0003/subSUB_ITEMLIST:SAPLMIGO:0200/tblSAPLMIGOTV_GOITEM/btnGOITEM-ZEILE[0,1]").press()
            
            screenshot.save(os.path.join(ss_folder, f"{nro_documento}-1.png"))
                    # Ítem 2
            session.findById("wnd[0]/usr/ssubSUB_MAIN_CARRIER:SAPLMIGO:0003/subSUB_ITEMLIST:SAPLMIGO:0200/tblSAPLMIGOTV_GOITEM").verticalScrollbar.position = 2
            session.findById("wnd[0]/usr/ssubSUB_MAIN_CARRIER:SAPLMIGO:0003/subSUB_ITEMLIST:SAPLMIGO:0200/tblSAPLMIGOTV_GOITEM/btnGOITEM-ZEILE[0,0]").setFocus()
            session.findById("wnd[0]/usr/ssubSUB_MAIN_CARRIER:SAPLMIGO:0003/subSUB_ITEMLIST:SAPLMIGO:0200/tblSAPLMIGOTV_GOITEM/btnGOITEM-ZEILE[0,0]").press()
            time.sleep(1)
            screenshot = pyautogui.screenshot()

            #Filtro para ver si es el ultimo ya alamos
            session.findById("wnd[0]/usr/ssubSUB_MAIN_CARRIER:SAPLMIGO:0003/subSUB_ITEMLIST:SAPLMIGO:0200/tblSAPLMIGOTV_GOITEM/btnGOITEM-ZEILE[0,1]").setFocus()
            session.findById("wnd[0]/usr/ssubSUB_MAIN_CARRIER:SAPLMIGO:0003/subSUB_ITEMLIST:SAPLMIGO:0200/tblSAPLMIGOTV_GOITEM/btnGOITEM-ZEILE[0,1]").press()
            
            screenshot.save(os.path.join(ss_folder, f"{nro_documento}-2.png"))
                        # Ítem 3
            session.findById("wnd[0]/usr/ssubSUB_MAIN_CARRIER:SAPLMIGO:0003/subSUB_ITEMLIST:SAPLMIGO:0200/tblSAPLMIGOTV_GOITEM").verticalScrollbar.position = 3
            session.findById("wnd[0]/usr/ssubSUB_MAIN_CARRIER:SAPLMIGO:0003/subSUB_ITEMLIST:SAPLMIGO:0200/tblSAPLMIGOTV_GOITEM/btnGOITEM-ZEILE[0,0]").setFocus()
            session.findById("wnd[0]/usr/ssubSUB_MAIN_CARRIER:SAPLMIGO:0003/subSUB_ITEMLIST:SAPLMIGO:0200/tblSAPLMIGOTV_GOITEM/btnGOITEM-ZEILE[0,0]").press()
            time.sleep(1)
            screenshot = pyautogui.screenshot()

            #Filtro para ver si es el ultimo ya alamos
            session.findById("wnd[0]/usr/ssubSUB_MAIN_CARRIER:SAPLMIGO:0003/subSUB_ITEMLIST:SAPLMIGO:0200/tblSAPLMIGOTV_GOITEM/btnGOITEM-ZEILE[0,1]").setFocus()
            session.findById("wnd[0]/usr/ssubSUB_MAIN_CARRIER:SAPLMIGO:0003/subSUB_ITEMLIST:SAPLMIGO:0200/tblSAPLMIGOTV_GOITEM/btnGOITEM-ZEILE[0,1]").press()
            
            screenshot.save(os.path.join(ss_folder, f"{nro_documento}-3.png"))
                        # Ítem 4
            session.findById("wnd[0]/usr/ssubSUB_MAIN_CARRIER:SAPLMIGO:0003/subSUB_ITEMLIST:SAPLMIGO:0200/tblSAPLMIGOTV_GOITEM").verticalScrollbar.position = 4
            time.sleep(3)
            session.findById("wnd[0]/usr/ssubSUB_MAIN_CARRIER:SAPLMIGO:0003/subSUB_ITEMLIST:SAPLMIGO:0200/tblSAPLMIGOTV_GOITEM/btnGOITEM-ZEILE[0,0]").setFocus()
            session.findById("wnd[0]/usr/ssubSUB_MAIN_CARRIER:SAPLMIGO:0003/subSUB_ITEMLIST:SAPLMIGO:0200/tblSAPLMIGOTV_GOITEM/btnGOITEM-ZEILE[0,0]").press()
            time.sleep(1)
            screenshot = pyautogui.screenshot()
            #Filtro para ver si es el ultimo ya alamos
            session.findById("wnd[0]/usr/ssubSUB_MAIN_CARRIER:SAPLMIGO:0003/subSUB_ITEMLIST:SAPLMIGO:0200/tblSAPLMIGOTV_GOITEM/btnGOITEM-ZEILE[0,1]").setFocus()
            session.findById("wnd[0]/usr/ssubSUB_MAIN_CARRIER:SAPLMIGO:0003/subSUB_ITEMLIST:SAPLMIGO:0200/tblSAPLMIGOTV_GOITEM/btnGOITEM-ZEILE[0,1]").press()

            screenshot.save(os.path.join(ss_folder, f"{nro_documento}-4.png"))
                        # Ítem 5
            session.findById("wnd[0]/usr/ssubSUB_MAIN_CARRIER:SAPLMIGO:0003/subSUB_ITEMLIST:SAPLMIGO:0200/tblSAPLMIGOTV_GOITEM").verticalScrollbar.position = 5
            session.findById("wnd[0]/usr/ssubSUB_MAIN_CARRIER:SAPLMIGO:0003/subSUB_ITEMLIST:SAPLMIGO:0200/tblSAPLMIGOTV_GOITEM/btnGOITEM-ZEILE[0,0]").setFocus()
            session.findById("wnd[0]/usr/ssubSUB_MAIN_CARRIER:SAPLMIGO:0003/subSUB_ITEMLIST:SAPLMIGO:0200/tblSAPLMIGOTV_GOITEM/btnGOITEM-ZEILE[0,0]").press()
            time.sleep(1)
            screenshot = pyautogui.screenshot()
            #Filtro para ver si es el ultimo ya alamos
            session.findById("wnd[0]/usr/ssubSUB_MAIN_CARRIER:SAPLMIGO:0003/subSUB_ITEMLIST:SAPLMIGO:0200/tblSAPLMIGOTV_GOITEM/btnGOITEM-ZEILE[0,1]").setFocus()
            session.findById("wnd[0]/usr/ssubSUB_MAIN_CARRIER:SAPLMIGO:0003/subSUB_ITEMLIST:SAPLMIGO:0200/tblSAPLMIGOTV_GOITEM/btnGOITEM-ZEILE[0,1]").press()
            
            screenshot.save(os.path.join(ss_folder, f"{nro_documento}-5.png"))
            # Ítem 6
            session.findById("wnd[0]/usr/ssubSUB_MAIN_CARRIER:SAPLMIGO:0003/subSUB_ITEMLIST:SAPLMIGO:0200/tblSAPLMIGOTV_GOITEM").verticalScrollbar.position = 6
            session.findById("wnd[0]/usr/ssubSUB_MAIN_CARRIER:SAPLMIGO:0003/subSUB_ITEMLIST:SAPLMIGO:0200/tblSAPLMIGOTV_GOITEM/btnGOITEM-ZEILE[0,0]").setFocus()
            session.findById("wnd[0]/usr/ssubSUB_MAIN_CARRIER:SAPLMIGO:0003/subSUB_ITEMLIST:SAPLMIGO:0200/tblSAPLMIGOTV_GOITEM/btnGOITEM-ZEILE[0,0]").press()
            time.sleep(1)
            screenshot = pyautogui.screenshot()
            #Filtro para ver si es el ultimo ya alamos
            session.findById("wnd[0]/usr/ssubSUB_MAIN_CARRIER:SAPLMIGO:0003/subSUB_ITEMLIST:SAPLMIGO:0200/tblSAPLMIGOTV_GOITEM/btnGOITEM-ZEILE[0,1]").setFocus()
            session.findById("wnd[0]/usr/ssubSUB_MAIN_CARRIER:SAPLMIGO:0003/subSUB_ITEMLIST:SAPLMIGO:0200/tblSAPLMIGOTV_GOITEM/btnGOITEM-ZEILE[0,1]").press()
            
            screenshot.save(os.path.join(ss_folder, f"{nro_documento}-6.png"))
                        # Ítem 7
            session.findById("wnd[0]/usr/ssubSUB_MAIN_CARRIER:SAPLMIGO:0003/subSUB_ITEMLIST:SAPLMIGO:0200/tblSAPLMIGOTV_GOITEM").verticalScrollbar.position = 7
            session.findById("wnd[0]/usr/ssubSUB_MAIN_CARRIER:SAPLMIGO:0003/subSUB_ITEMLIST:SAPLMIGO:0200/tblSAPLMIGOTV_GOITEM/btnGOITEM-ZEILE[0,0]").setFocus()
            session.findById("wnd[0]/usr/ssubSUB_MAIN_CARRIER:SAPLMIGO:0003/subSUB_ITEMLIST:SAPLMIGO:0200/tblSAPLMIGOTV_GOITEM/btnGOITEM-ZEILE[0,0]").press()
            time.sleep(1)
            screenshot = pyautogui.screenshot()
            #Filtro para ver si es el ultimo ya alamos
            session.findById("wnd[0]/usr/ssubSUB_MAIN_CARRIER:SAPLMIGO:0003/subSUB_ITEMLIST:SAPLMIGO:0200/tblSAPLMIGOTV_GOITEM/btnGOITEM-ZEILE[0,1]").setFocus()
            session.findById("wnd[0]/usr/ssubSUB_MAIN_CARRIER:SAPLMIGO:0003/subSUB_ITEMLIST:SAPLMIGO:0200/tblSAPLMIGOTV_GOITEM/btnGOITEM-ZEILE[0,1]").press()

            screenshot.save(os.path.join(ss_folder, f"{nro_documento}-7.png"))
                    # Ítem 2
            session.findById("wnd[0]/usr/ssubSUB_MAIN_CARRIER:SAPLMIGO:0003/subSUB_ITEMLIST:SAPLMIGO:0200/tblSAPLMIGOTV_GOITEM").verticalScrollbar.position = 8
            time.sleep(3)
            session.findById("wnd[0]/usr/ssubSUB_MAIN_CARRIER:SAPLMIGO:0003/subSUB_ITEMLIST:SAPLMIGO:0200/tblSAPLMIGOTV_GOITEM/btnGOITEM-ZEILE[0,0]").setFocus()
            session.findById("wnd[0]/usr/ssubSUB_MAIN_CARRIER:SAPLMIGO:0003/subSUB_ITEMLIST:SAPLMIGO:0200/tblSAPLMIGOTV_GOITEM/btnGOITEM-ZEILE[0,0]").press()
            time.sleep(1)
            screenshot = pyautogui.screenshot()
            #Filtro para ver si es el ultimo ya alamos
            session.findById("wnd[0]/usr/ssubSUB_MAIN_CARRIER:SAPLMIGO:0003/subSUB_ITEMLIST:SAPLMIGO:0200/tblSAPLMIGOTV_GOITEM/btnGOITEM-ZEILE[0,1]").setFocus()
            session.findById("wnd[0]/usr/ssubSUB_MAIN_CARRIER:SAPLMIGO:0003/subSUB_ITEMLIST:SAPLMIGO:0200/tblSAPLMIGOTV_GOITEM/btnGOITEM-ZEILE[0,1]").press()

            screenshot.save(os.path.join(ss_folder, f"{nro_documento}-8.png"))
                        # Ítem 3
            session.findById("wnd[0]/usr/ssubSUB_MAIN_CARRIER:SAPLMIGO:0003/subSUB_ITEMLIST:SAPLMIGO:0200/tblSAPLMIGOTV_GOITEM").verticalScrollbar.position = 9
            session.findById("wnd[0]/usr/ssubSUB_MAIN_CARRIER:SAPLMIGO:0003/subSUB_ITEMLIST:SAPLMIGO:0200/tblSAPLMIGOTV_GOITEM/btnGOITEM-ZEILE[0,0]").setFocus()
            session.findById("wnd[0]/usr/ssubSUB_MAIN_CARRIER:SAPLMIGO:0003/subSUB_ITEMLIST:SAPLMIGO:0200/tblSAPLMIGOTV_GOITEM/btnGOITEM-ZEILE[0,0]").press()
            time.sleep(1)
            screenshot = pyautogui.screenshot()
            #Filtro para ver si es el ultimo ya alamos
            session.findById("wnd[0]/usr/ssubSUB_MAIN_CARRIER:SAPLMIGO:0003/subSUB_ITEMLIST:SAPLMIGO:0200/tblSAPLMIGOTV_GOITEM/btnGOITEM-ZEILE[0,1]").setFocus()
            session.findById("wnd[0]/usr/ssubSUB_MAIN_CARRIER:SAPLMIGO:0003/subSUB_ITEMLIST:SAPLMIGO:0200/tblSAPLMIGOTV_GOITEM/btnGOITEM-ZEILE[0,1]").press()

            screenshot.save(os.path.join(ss_folder, f"{nro_documento}-9.png"))
                        # Ítem 4
            session.findById("wnd[0]/usr/ssubSUB_MAIN_CARRIER:SAPLMIGO:0003/subSUB_ITEMLIST:SAPLMIGO:0200/tblSAPLMIGOTV_GOITEM").verticalScrollbar.position = 10
            session.findById("wnd[0]/usr/ssubSUB_MAIN_CARRIER:SAPLMIGO:0003/subSUB_ITEMLIST:SAPLMIGO:0200/tblSAPLMIGOTV_GOITEM/btnGOITEM-ZEILE[0,0]").setFocus()
            session.findById("wnd[0]/usr/ssubSUB_MAIN_CARRIER:SAPLMIGO:0003/subSUB_ITEMLIST:SAPLMIGO:0200/tblSAPLMIGOTV_GOITEM/btnGOITEM-ZEILE[0,0]").press()
            time.sleep(1)
            screenshot = pyautogui.screenshot()
            
            #Filtro para ver si es el ultimo ya alamos
            session.findById("wnd[0]/usr/ssubSUB_MAIN_CARRIER:SAPLMIGO:0003/subSUB_ITEMLIST:SAPLMIGO:0200/tblSAPLMIGOTV_GOITEM/btnGOITEM-ZEILE[0,1]").setFocus()
            session.findById("wnd[0]/usr/ssubSUB_MAIN_CARRIER:SAPLMIGO:0003/subSUB_ITEMLIST:SAPLMIGO:0200/tblSAPLMIGOTV_GOITEM/btnGOITEM-ZEILE[0,1]").press()

            screenshot.save(os.path.join(ss_folder, f"{nro_documento}-10.png"))
                        # Ítem 5
            session.findById("wnd[0]/usr/ssubSUB_MAIN_CARRIER:SAPLMIGO:0003/subSUB_ITEMLIST:SAPLMIGO:0200/tblSAPLMIGOTV_GOITEM").verticalScrollbar.position = 11
            session.findById("wnd[0]/usr/ssubSUB_MAIN_CARRIER:SAPLMIGO:0003/subSUB_ITEMLIST:SAPLMIGO:0200/tblSAPLMIGOTV_GOITEM/btnGOITEM-ZEILE[0,0]").setFocus()
            session.findById("wnd[0]/usr/ssubSUB_MAIN_CARRIER:SAPLMIGO:0003/subSUB_ITEMLIST:SAPLMIGO:0200/tblSAPLMIGOTV_GOITEM/btnGOITEM-ZEILE[0,0]").press()
            time.sleep(1)
            screenshot = pyautogui.screenshot()
            #Filtro para ver si es el ultimo ya alamos
            session.findById("wnd[0]/usr/ssubSUB_MAIN_CARRIER:SAPLMIGO:0003/subSUB_ITEMLIST:SAPLMIGO:0200/tblSAPLMIGOTV_GOITEM/btnGOITEM-ZEILE[0,1]").setFocus()
            session.findById("wnd[0]/usr/ssubSUB_MAIN_CARRIER:SAPLMIGO:0003/subSUB_ITEMLIST:SAPLMIGO:0200/tblSAPLMIGOTV_GOITEM/btnGOITEM-ZEILE[0,1]").press()
            screenshot.save(os.path.join(ss_folder, f"{nro_documento}-11.png"))

        except Exception as e:
            print(f"Error o fin de ítems en el ítem 0 del documento {nro_documento}: {str(e)}")

    
    except Exception as e:
        # En caso de error, almacenar el error en la columna final
        ws.cell(row=i, column=ws.max_column).value = f"Error: {str(e)}"

# Finalmente, guardar el archivo Excel con los cambios
wb.save("ruta_al_archivo.xlsx")


In [22]:
# Cargar archivo de asientos
archivo = r"C:\Users\Ext_rgutierrezc\OneDrive - Grupo Alicorp\Escritorio\Siglo BPO (1)\Base2.xlsx"
wb = load_workbook(archivo)
ws=wb['DATA']
sociedad="PE18"

# Definir la ruta de la carpeta donde se guardarán las capturas de pantalla
base_path = r"C:\Users\Ext_rgutierrezc\OneDrive - Grupo Alicorp\Escritorio\Siglo BPO (1)"
ss_folder = os.path.join(base_path, "SCREENSHOTS - SEGUNDA CAPTURA - MILAGROS")

# Crear la carpeta "SS" si no existe
if not os.path.exists(ss_folder):
    os.makedirs(ss_folder)

In [30]:
session = get_session()

for i, fila in enumerate(ws.iter_rows(min_row=2, max_row=ws.max_row, values_only=True), start=2):
    # Solo filas con datos
    nro_documento = fila[0]
    sociedad="PE18"
    if not nro_documento:
        continue

    try:
        ejercicio = 2022
        # Ir a la TX 
        enter_tx("FB03")
        session.findById("wnd[0]").maximize()

        # Nro Documento
        session.findById("wnd[0]/usr/txtRF05L-BELNR").text = nro_documento
        # Sociedad
        session.findById("wnd[0]/usr/ctxtRF05L-BUKRS").text = sociedad
        # Ejercicio
        session.findById("wnd[0]/usr/txtRF05L-GJAHR").text = ejercicio
        # Ejecutar
        session.findById("wnd[0]").sendVKey(0)

        ### SOMBRERITO

        session.findById("wnd[0]/tbar[1]/btn[5]").press()
        session.findById("wnd[1]/usr/txtBKPF-AWKEY").setFocus()
        session.findById("wnd[1]/usr/txtBKPF-AWKEY").caretPosition = 14
        session.findById("wnd[1]").sendVKey(2)

        # SELECCIONAR CANTIDADES
        session.findById("wnd[0]/usr/ssubSUB_MAIN_CARRIER:SAPLMIGO:0003/subSUB_ITEMDETAIL:SAPLMIGO:0301/subSUB_DETAIL:SAPLMIGO:0300/tabsTS_GOITEM/tabpOK_GOITEM_QUANTITIES").select()

        # Intentamos seleccionar los ítems manualmente uno por uno
        try:
            # Ítem 0
            session.findById("wnd[0]/usr/ssubSUB_MAIN_CARRIER:SAPLMIGO:0003/subSUB_ITEMLIST:SAPLMIGO:0200/tblSAPLMIGOTV_GOITEM").columns.elementAt(2).selected = True
            session.findById("wnd[0]/usr/ssubSUB_MAIN_CARRIER:SAPLMIGO:0003/subSUB_ITEMLIST:SAPLMIGO:0200/tblSAPLMIGOTV_GOITEM").reorderTable("0 45 1 46 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77")
            session.findById("wnd[0]/usr/ssubSUB_MAIN_CARRIER:SAPLMIGO:0003/subSUB_ITEMLIST:SAPLMIGO:0200/tblSAPLMIGOTV_GOITEM").columns.elementAt(2).width = 38
            session.findById("wnd[0]/usr/ssubSUB_MAIN_CARRIER:SAPLMIGO:0003/subSUB_ITEMLIST:SAPLMIGO:0200/tblSAPLMIGOTV_GOITEM").columns.elementAt(1).width = 11

            session.findById("wnd[0]/usr/ssubSUB_MAIN_CARRIER:SAPLMIGO:0003/subSUB_ITEMLIST:SAPLMIGO:0200/tblSAPLMIGOTV_GOITEM/btnGOITEM-ZEILE[0,0]").setFocus()
            session.findById("wnd[0]/usr/ssubSUB_MAIN_CARRIER:SAPLMIGO:0003/subSUB_ITEMLIST:SAPLMIGO:0200/tblSAPLMIGOTV_GOITEM/btnGOITEM-ZEILE[0,0]").press()
            time.sleep(1)
            screenshot = pyautogui.screenshot()
            
            screenshot.save(os.path.join(ss_folder, f"{nro_documento}-0.png"))
                        # Ítem 1
            session.findById("wnd[0]/usr/ssubSUB_MAIN_CARRIER:SAPLMIGO:0003/subSUB_ITEMLIST:SAPLMIGO:0200/tblSAPLMIGOTV_GOITEM").verticalScrollbar.position = 1
            
            session.findById("wnd[0]/usr/ssubSUB_MAIN_CARRIER:SAPLMIGO:0003/subSUB_ITEMLIST:SAPLMIGO:0200/tblSAPLMIGOTV_GOITEM/btnGOITEM-ZEILE[0,0]").setFocus()
            session.findById("wnd[0]/usr/ssubSUB_MAIN_CARRIER:SAPLMIGO:0003/subSUB_ITEMLIST:SAPLMIGO:0200/tblSAPLMIGOTV_GOITEM/btnGOITEM-ZEILE[0,0]").press()
            time.sleep(1)
            screenshot = pyautogui.screenshot()
             
            
            screenshot.save(os.path.join(ss_folder, f"{nro_documento}-1.png"))
                    # Ítem 2
            session.findById("wnd[0]/usr/ssubSUB_MAIN_CARRIER:SAPLMIGO:0003/subSUB_ITEMLIST:SAPLMIGO:0200/tblSAPLMIGOTV_GOITEM").verticalScrollbar.position = 2
            session.findById("wnd[0]/usr/ssubSUB_MAIN_CARRIER:SAPLMIGO:0003/subSUB_ITEMLIST:SAPLMIGO:0200/tblSAPLMIGOTV_GOITEM/btnGOITEM-ZEILE[0,0]").setFocus()
            session.findById("wnd[0]/usr/ssubSUB_MAIN_CARRIER:SAPLMIGO:0003/subSUB_ITEMLIST:SAPLMIGO:0200/tblSAPLMIGOTV_GOITEM/btnGOITEM-ZEILE[0,0]").press()
            time.sleep(1)
            screenshot = pyautogui.screenshot()

            #Filtro para ver si es el ultimo ya alamos

            screenshot.save(os.path.join(ss_folder, f"{nro_documento}-2.png"))
                        # Ítem 3
            session.findById("wnd[0]/usr/ssubSUB_MAIN_CARRIER:SAPLMIGO:0003/subSUB_ITEMLIST:SAPLMIGO:0200/tblSAPLMIGOTV_GOITEM").verticalScrollbar.position = 3
            session.findById("wnd[0]/usr/ssubSUB_MAIN_CARRIER:SAPLMIGO:0003/subSUB_ITEMLIST:SAPLMIGO:0200/tblSAPLMIGOTV_GOITEM/btnGOITEM-ZEILE[0,0]").setFocus()
            session.findById("wnd[0]/usr/ssubSUB_MAIN_CARRIER:SAPLMIGO:0003/subSUB_ITEMLIST:SAPLMIGO:0200/tblSAPLMIGOTV_GOITEM/btnGOITEM-ZEILE[0,0]").press()
            time.sleep(1)
            screenshot = pyautogui.screenshot()

            #Filtro para ver si es el ultimo ya alamos
            
            screenshot.save(os.path.join(ss_folder, f"{nro_documento}-3.png"))
                        # Ítem 4
            session.findById("wnd[0]/usr/ssubSUB_MAIN_CARRIER:SAPLMIGO:0003/subSUB_ITEMLIST:SAPLMIGO:0200/tblSAPLMIGOTV_GOITEM").verticalScrollbar.position = 4
            session.findById("wnd[0]/usr/ssubSUB_MAIN_CARRIER:SAPLMIGO:0003/subSUB_ITEMLIST:SAPLMIGO:0200/tblSAPLMIGOTV_GOITEM/btnGOITEM-ZEILE[0,0]").setFocus()
            session.findById("wnd[0]/usr/ssubSUB_MAIN_CARRIER:SAPLMIGO:0003/subSUB_ITEMLIST:SAPLMIGO:0200/tblSAPLMIGOTV_GOITEM/btnGOITEM-ZEILE[0,0]").press()
            time.sleep(1)
            screenshot = pyautogui.screenshot()
            #Filtro para ver si es el ultimo ya alamos

            screenshot.save(os.path.join(ss_folder, f"{nro_documento}-4.png"))
                        # Ítem 5
            session.findById("wnd[0]/usr/ssubSUB_MAIN_CARRIER:SAPLMIGO:0003/subSUB_ITEMLIST:SAPLMIGO:0200/tblSAPLMIGOTV_GOITEM").verticalScrollbar.position = 5
            session.findById("wnd[0]/usr/ssubSUB_MAIN_CARRIER:SAPLMIGO:0003/subSUB_ITEMLIST:SAPLMIGO:0200/tblSAPLMIGOTV_GOITEM/btnGOITEM-ZEILE[0,0]").setFocus()
            session.findById("wnd[0]/usr/ssubSUB_MAIN_CARRIER:SAPLMIGO:0003/subSUB_ITEMLIST:SAPLMIGO:0200/tblSAPLMIGOTV_GOITEM/btnGOITEM-ZEILE[0,0]").press()
            time.sleep(1)
            screenshot = pyautogui.screenshot()
            #Filtro para ver si es el ultimo ya alamos

            
            screenshot.save(os.path.join(ss_folder, f"{nro_documento}-5.png"))
            # Ítem 6
            session.findById("wnd[0]/usr/ssubSUB_MAIN_CARRIER:SAPLMIGO:0003/subSUB_ITEMLIST:SAPLMIGO:0200/tblSAPLMIGOTV_GOITEM").verticalScrollbar.position = 6
            session.findById("wnd[0]/usr/ssubSUB_MAIN_CARRIER:SAPLMIGO:0003/subSUB_ITEMLIST:SAPLMIGO:0200/tblSAPLMIGOTV_GOITEM/btnGOITEM-ZEILE[0,0]").setFocus()
            session.findById("wnd[0]/usr/ssubSUB_MAIN_CARRIER:SAPLMIGO:0003/subSUB_ITEMLIST:SAPLMIGO:0200/tblSAPLMIGOTV_GOITEM/btnGOITEM-ZEILE[0,0]").press()
            time.sleep(1)
            screenshot = pyautogui.screenshot()
            #Filtro para ver si es el ultimo ya alamos
            
            screenshot.save(os.path.join(ss_folder, f"{nro_documento}-6.png"))
                        # Ítem 7
            session.findById("wnd[0]/usr/ssubSUB_MAIN_CARRIER:SAPLMIGO:0003/subSUB_ITEMLIST:SAPLMIGO:0200/tblSAPLMIGOTV_GOITEM").verticalScrollbar.position = 7
            session.findById("wnd[0]/usr/ssubSUB_MAIN_CARRIER:SAPLMIGO:0003/subSUB_ITEMLIST:SAPLMIGO:0200/tblSAPLMIGOTV_GOITEM/btnGOITEM-ZEILE[0,0]").setFocus()
            session.findById("wnd[0]/usr/ssubSUB_MAIN_CARRIER:SAPLMIGO:0003/subSUB_ITEMLIST:SAPLMIGO:0200/tblSAPLMIGOTV_GOITEM/btnGOITEM-ZEILE[0,0]").press()
            time.sleep(1)
            screenshot = pyautogui.screenshot()
            #Filtro para ver si es el ultimo ya alamos


            screenshot.save(os.path.join(ss_folder, f"{nro_documento}-7.png"))
                    # Ítem 2
            session.findById("wnd[0]/usr/ssubSUB_MAIN_CARRIER:SAPLMIGO:0003/subSUB_ITEMLIST:SAPLMIGO:0200/tblSAPLMIGOTV_GOITEM").verticalScrollbar.position = 8
            session.findById("wnd[0]/usr/ssubSUB_MAIN_CARRIER:SAPLMIGO:0003/subSUB_ITEMLIST:SAPLMIGO:0200/tblSAPLMIGOTV_GOITEM/btnGOITEM-ZEILE[0,0]").setFocus()
            session.findById("wnd[0]/usr/ssubSUB_MAIN_CARRIER:SAPLMIGO:0003/subSUB_ITEMLIST:SAPLMIGO:0200/tblSAPLMIGOTV_GOITEM/btnGOITEM-ZEILE[0,0]").press()
            time.sleep(1)
            screenshot = pyautogui.screenshot()
            #Filtro para ver si es el ultimo ya alamos


            screenshot.save(os.path.join(ss_folder, f"{nro_documento}-8.png"))
                        # Ítem 3
            session.findById("wnd[0]/usr/ssubSUB_MAIN_CARRIER:SAPLMIGO:0003/subSUB_ITEMLIST:SAPLMIGO:0200/tblSAPLMIGOTV_GOITEM").verticalScrollbar.position = 9
            session.findById("wnd[0]/usr/ssubSUB_MAIN_CARRIER:SAPLMIGO:0003/subSUB_ITEMLIST:SAPLMIGO:0200/tblSAPLMIGOTV_GOITEM/btnGOITEM-ZEILE[0,0]").setFocus()
            session.findById("wnd[0]/usr/ssubSUB_MAIN_CARRIER:SAPLMIGO:0003/subSUB_ITEMLIST:SAPLMIGO:0200/tblSAPLMIGOTV_GOITEM/btnGOITEM-ZEILE[0,0]").press()
            time.sleep(1)
            screenshot = pyautogui.screenshot()
            #Filtro para ver si es el ultimo ya alamos


            screenshot.save(os.path.join(ss_folder, f"{nro_documento}-9.png"))
                        # Ítem 4
            session.findById("wnd[0]/usr/ssubSUB_MAIN_CARRIER:SAPLMIGO:0003/subSUB_ITEMLIST:SAPLMIGO:0200/tblSAPLMIGOTV_GOITEM").verticalScrollbar.position = 10
            session.findById("wnd[0]/usr/ssubSUB_MAIN_CARRIER:SAPLMIGO:0003/subSUB_ITEMLIST:SAPLMIGO:0200/tblSAPLMIGOTV_GOITEM/btnGOITEM-ZEILE[0,0]").setFocus()
            session.findById("wnd[0]/usr/ssubSUB_MAIN_CARRIER:SAPLMIGO:0003/subSUB_ITEMLIST:SAPLMIGO:0200/tblSAPLMIGOTV_GOITEM/btnGOITEM-ZEILE[0,0]").press()
            time.sleep(1)
            screenshot = pyautogui.screenshot()
            
            #Filtro para ver si es el ultimo ya alamos

            screenshot.save(os.path.join(ss_folder, f"{nro_documento}-10.png"))
                        # Ítem 5
            session.findById("wnd[0]/usr/ssubSUB_MAIN_CARRIER:SAPLMIGO:0003/subSUB_ITEMLIST:SAPLMIGO:0200/tblSAPLMIGOTV_GOITEM").verticalScrollbar.position = 11
            session.findById("wnd[0]/usr/ssubSUB_MAIN_CARRIER:SAPLMIGO:0003/subSUB_ITEMLIST:SAPLMIGO:0200/tblSAPLMIGOTV_GOITEM/btnGOITEM-ZEILE[0,0]").setFocus()
            session.findById("wnd[0]/usr/ssubSUB_MAIN_CARRIER:SAPLMIGO:0003/subSUB_ITEMLIST:SAPLMIGO:0200/tblSAPLMIGOTV_GOITEM/btnGOITEM-ZEILE[0,0]").press()
            time.sleep(1)
            screenshot = pyautogui.screenshot()

            screenshot.save(os.path.join(ss_folder, f"{nro_documento}-11.png"))

        except Exception as e:
            print(f"Error o fin de ítems en el ítem 0 del documento {nro_documento}: {str(e)}")

    
    except Exception as e:
        # En caso de error, almacenar el error en la columna final
        ws.cell(row=i, column=ws.max_column).value = f"Error: {str(e)}"

# Finalmente, guardar el archivo Excel con los cambios
wb.save("ruta_al_archivo.xlsx")
